# 1. Fyers Documentation

In [ ]:
Contract Type      Quantity (Contract Size)   Price Quoted As   Typical Use
----------------------------------------------------------------------------
Gold               1 KG (1000 g)              ₹ per 10 grams    Hedging / High value
Gold Mini          100 g                      ₹ per 10 grams    Retail / Lower margin
Silver             30 KG                      ₹ per KG          Hedging / Institutions
Silver Mini        5 KG                       ₹ per KG          Retail
Silver Micro       1 KG                       ₹ per KG          Small traders



Gold       : Tick size 1 → ₹100 per contract
Gold Mini  : Tick size 1 → ₹10 per contract

Silver     : Tick size 1 → ₹30 per contract
Silver Mini: Tick size 1 → ₹5 per contract
Silver Micro: Tick size 1 → ₹1 per contract

# 2. Setup Fyers Clients

## 2.1 Generate Auth Code

In [5]:
# Constants
from datetime import datetime, date

GOLD_MINI = "GOLDM"
SILVER_MINI = "SILVERM"
GOLD_MINI_MULTIPLIER = 10
SILVER_MINI_MULTIPLIER = 5
TODAY = date.today()

In [6]:
from urllib.parse import urlparse, parse_qs
from fyers_apiv3 import fyersModel
import pandas as pd
from io import StringIO
import webbrowser
import json
import os

redirect_uri= "https://kite.zerodha.com/markets"  ## redircet_uri you entered while creating APP.
CLIENT_ID = "LLSNJOGT4J-100"                       ## Client_id here refers to APP_ID of the created app
secret_key = "P8R047KG4J"                          ## app_secret key which you got after creating the app 
grant_type = "authorization_code"                  ## The grant_type always has to be "authorization_code"
response_type = "code"                             ## The response_type always has to be "code"
state = "sample"                                   ##  The state field here acts as a session manager. you will be sent with the state field after successfull generation of auth_code 

appSession = fyersModel.SessionModel(client_id = CLIENT_ID, redirect_uri = redirect_uri,response_type=response_type,state=state,secret_key=secret_key,grant_type=grant_type)

# ## Make  a request to generate_authcode object this will return a login url which you need to open in your browser from where you can get the generated auth_code 
generateTokenUrl = appSession.generate_authcode()
print("Auth Token Url:   ", generateTokenUrl)

webbrowser.open(generateTokenUrl,new=0)


Auth Token Url:    https://api-t1.fyers.in/api/v3/generate-authcode?client_id=LLSNJOGT4J-100&redirect_uri=https%3A%2F%2Fkite.zerodha.com%2Fmarkets&response_type=code&state=sample


True

## 2.2 Paste Auth Code

In [3]:

AUTH_TOKEN_SUCCESS_URL = "https://kite.zerodha.com/markets?s=ok&code=200&auth_code=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhcHBfaWQiOiJMTFNOSk9HVDRKIiwidXVpZCI6IjMwMmRmYTNlOWJhNzQ2NGRiZTBjMTM3NjdmNGRlNTM1IiwiaXBBZGRyIjoiIiwibm9uY2UiOiIiLCJzY29wZSI6IiIsImRpc3BsYXlfbmFtZSI6IlhTMTk5ODUiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJlZDRhYWE2YWYxMDVjY2UxZGFjN2Y4ODQ3Nzg1MWY3YTdhNjM3NWU1NTZiN2NlMTkxMjFmYzNiOSIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImF1ZCI6IltcImQ6MVwiLFwiZDoyXCIsXCJ4OjBcIixcIng6MVwiXSIsImV4cCI6MTc2OTMxMjMxNiwiaWF0IjoxNzY5MjgyMzE2LCJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJuYmYiOjE3NjkyODIzMTYsInN1YiI6ImF1dGhfY29kZSJ9.8wvMQJTP5nRNYPbIoy-INX33adf41yj7BwtKSLHoCTM&state=sample"

parsed_url = urlparse(AUTH_TOKEN_SUCCESS_URL)
query_params = parse_qs(parsed_url.query)
auth_code = query_params.get("auth_code", [None])[0]

appSession.set_token(auth_code)
response = appSession.generate_token()

ACCESS_TOKEN = None
try: 
    ACCESS_TOKEN = response["access_token"]
except Exception as e:
    print("Can not generate Token",response)  ## This will help you in debugging then and there itself like what was the error and also you would be able to see the value you got in response variable. instead of getting key_error for unsuccessfull response.

print("Access Token: ", ACCESS_TOKEN)

Access Token:  eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIl0sImF0X2hhc2giOiJnQUFBQUFCcGRSc2JDbjlpNDZoMVFuNU02VEY2VWQ1UUF4X2QxRnpTLXFOUWZobmpDN2VCUDFyOFh6UEZXamd5VWU4WENoYkk2U2pBeWZvbVNvTUF6OVlYZWpvajJTYi1oNFZUQWdDV2ZHN2dYMnVrRjdPVXFtWT0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJlZDRhYWE2YWYxMDVjY2UxZGFjN2Y4ODQ3Nzg1MWY3YTdhNjM3NWU1NTZiN2NlMTkxMjFmYzNiOSIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWFMxOTk4NSIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzY5Mzg3NDAwLCJpYXQiOjE3NjkyODIzMzEsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc2OTI4MjMzMSwic3ViIjoiYWNjZXNzX3Rva2VuIn0.5U3d1zNbYuxiaX3yyyzzUlq-Qthhx6xaq0EsFRT6f6I


## 2.3 Fyers Client

In [4]:
class FyersClient:
    def __init__(self):
        self.client_id = CLIENT_ID
        self.auth_token = ACCESS_TOKEN
        self.fyers = self._load_fyers()

    def _load_fyers(self):
        fyers_client = fyersModel.FyersModel(
            client_id=self.client_id,
            token=self.auth_token,
            log_path=""
        )
        return fyers_client

    def get_quotes(self, symbols):
        """
        Returns LTP, OHLC, volume, expiry-related info
        symbols = [
            "MCX:GOLDM26JANFUT",
            "MCX:GOLDM26FEBFUT",
        ]
        """

        data = {
            "symbols": ",".join(symbols)
        }

        response = self.fyers.quotes(data)
        return response

fyers = FyersClient()        
fyers

## 2.4 Setup MCX Client to get Master MCX File (Symbols only)

In [2]:
import pandas as pd

MCX_CONTRACT_MASTER_URL = "https://public.fyers.in/sym_details/MCX_COM.csv"
COLUMNS = [
    "fytoken",
    "symbol_details",
    "exchange_instrument_type",
    "min_lot_size",
    "tick_size",
    "isin",
    "trading_session",
    "last_update_date",
    "expiry_epoch",
    "exchange_symbol",
    "exchange",
    "segment",
    "scrip_code",
    "underlying_symbol",
    "underlying_scrip_code",
    "strike_price",
    "option_type",
    "underlying_fytoken",
    "expiry_epoch_dup",
    "reserved_1",
    "reserved_2"
]

def get_active_future_symbol_df(symbol):
    """
        symbol: "goldm"
    """
    raw = pd.read_csv(MCX_CONTRACT_MASTER_URL, header=None, names=COLUMNS, dtype=str)
    print(raw.head())
    raw.to_csv("test.csv")
    filtered_df = raw[raw["underlying_symbol"] == symbol]
    filtered_df = filtered_df[filtered_df["symbol_details"].str.contains("fut", case=False, na=False)]    
    return filtered_df

get_active_future_symbol_df("abc")


            fytoken            symbol_details exchange_instrument_type  \
0  1120260127486243  MCXBULLDEX 26 Jan 27 FUT                       11   
1  1120260224488289  MCXBULLDEX 26 Feb 24 FUT                       11   
2  1120260324499305  MCXBULLDEX 26 Mar 24 FUT                       11   
3  1120260424521034  MCXBULLDEX 26 Apr 24 FUT                       11   
4  1120260220488282  MCXMETLDEX 26 Feb 20 FUT                       11   

  min_lot_size tick_size isin       trading_session last_update_date  \
0            1       1.0  NaN  0900-2355|1815-1915:       2026-01-24   
1            1       1.0  NaN  0900-2355|1815-1915:       2026-01-24   
2            1       1.0  NaN  0900-2355|1815-1915:       2026-01-24   
3            1       1.0  NaN  0900-2355|1815-1915:       2026-01-24   
4            1       1.0  NaN  0900-2355|1815-1915:       2026-01-24   

  expiry_epoch         exchange_symbol  ... segment scrip_code  \
0   1769538300  MCX:MCXBULLDEX26JANFUT  ...      20     

,fytoken,symbol_details,exchange_instrument_type,min_lot_size,tick_size,isin,trading_session,last_update_date,expiry_epoch,exchange_symbol,...,segment,scrip_code,underlying_symbol,underlying_scrip_code,strike_price,option_type,underlying_fytoken,expiry_epoch_dup,reserved_1,reserved_2


In [ ]:
import os
import pandas as pd
from typing import List, Dict


class FyersMasterClient:
    """
    Manages Fyers MCX contract master using
    a local Excel cache as the source of truth.
    """

    MCX_CONTRACT_MASTER_URL = "https://public.fyers.in/sym_details/MCX_COM.csv"
    MCX_CONTRACT_MASTER_FILE = "mcx_fyers_master.xlsx"
    
    COLUMNS = [
        "fytoken",
        "symbol_details",
        "exchange_instrument_type",
        "min_lot_size",
        "tick_size",
        "isin",
        "trading_session",
        "last_update_date",
        "expiry_epoch",
        "exchange_symbol",
        "exchange",
        "segment",
        "scrip_code",
        "underlying_symbol",
        "underlying_scrip_code",
        "strike_price",
        "option_type",
        "underlying_fytoken",
        "expiry_epoch_dup",
        "reserved_1",
        "reserved_2",
    ]

    def __init__(self):
        self._cache_file = MCX_CONTRACT_MASTER_FILE

    # ==================================================
    # Public APIs
    # ==================================================

    def update_fyers_master_cache_file(self) -> None:
        """
        Refreshes the entire master file:
        - Deletes existing cache
        - Downloads full CSV from API
        - Saves as Excel
        """
        if os.path.exists(self._cache_file):
            os.remove(self._cache_file)

        df = pd.read_csv(
            self.MCX_CONTRACT_MASTER_URL,
            header=None,
            names=self.COLUMNS,
            dtype=str,
        )

        df.to_excel(self._cache_file, index=False)

    def get_symbol_master_details(self, symbols: List[str]) -> Dict[str, dict]:
        """
        Exact match on exchange_symbol.
        Reads only from cache.
        """
        df = self._load_cache()
        result: Dict[str, dict] = {}

        symbols = [s.lower() for s in symbols]

        for symbol in symbols:
            symbol_df = df[
                df["exchange_symbol"].str.lower() == symbol
            ]

            if not symbol_df.empty:
                result[symbol] = self._select_active_future(symbol_df)
            else:
                result[symbol] = {}

        return result

    def get_symbol_master_details_by_underlying_symbol(
        self, underlying_symbol: str
    ) -> dict:
        """
        Exact match on underlying_symbol.
        Only FUT contracts.
        Reads only from cache.
        """
        df = self._load_cache()

        filtered_df = df[
            (df["underlying_symbol"].str.lower() == underlying_symbol.lower()) &
            (df["symbol_details"].str.contains("fut", case=False, na=False))
        ]

        return self._select_active_future(filtered_df)

    # ==================================================
    # Internal Helpers
    # ==================================================

    def _load_cache(self) -> pd.DataFrame:
        if not os.path.exists(self._cache_file):
            raise FileNotFoundError(
                "Fyers master cache file not found. "
                "Call update_fyers_master_cache_file() first."
            )

        return pd.read_excel(self._cache_file, dtype=str)

    def _select_active_future(self, df: pd.DataFrame) -> dict:
        """
        Selects nearest expiry contract.
        """
        if df.empty:
            return {}

        df = df.copy()
        df["expiry_epoch"] = pd.to_numeric(df["expiry_epoch"], errors="coerce")

        active_row = df.sort_values("expiry_epoch").iloc[0]
        return active_row.to_dict()

## 2.5 Get Combined Future DF

In [6]:
def get_combined_future_price_df(future_symbol_df):
    """
        Input: 
            - future_symbol_df: MCX futures DataFrame
        
        Output: 
            - futures_price_df: futures_price_df
            - current_month_future_symbol: Active month future symbol, should be less than 30 days for expiry.
    """
    print("List of Future Symbols: ", list(future_symbol_df["exchange_symbol"]))

    future_price_data = fyers.get_quotes(list(future_symbol_df["exchange_symbol"]))

    # Current month future
    current_month_future = future_price_data["d"][0]["v"]

    # Current month future Validation
    if (current_month_future["symbol"] != future_symbol_df.loc[future_symbol_df["expiry_epoch"].idxmin(), "exchange_symbol"]):
        raise ValueError("Current Month Future details are not available.")
    
    rows = []
    for item_raw in future_price_data["d"]:
        """ Notes: To get all rows, print item.
        """
        item = item_raw["v"]
        expiry_epoch = future_symbol_df.loc[future_symbol_df["exchange_symbol"] == item["symbol"], "expiry_epoch"].iloc[0]    
        expiry_date = datetime.fromtimestamp(int(expiry_epoch)).date().strftime("%d-%m-%Y")
    
        days_left = (datetime.strptime(expiry_date, "%d-%m-%Y").date() - TODAY).days
                
        row = {
            "symbol": item["symbol"],
            "expiry": expiry_date,
            "price": item["lp"],
    
            "days_left": days_left,
    
            "price_yesterday": item["prev_close_price"],
            "volume": item["volume"],
            "spread": item["spread"],
            "price_change_percentage": item["chp"],
        }
        rows.append(row)

    return (pd.DataFrame(rows), current_month_future["symbol"])

def add_analysis_per_day_interest_details(future_price_df, current_month_symbol, multiplier):
    """
        Input:

            - multipler: Example gold mini multiplier is 10
    """

    current_month_df = future_price_df[future_price_df["symbol"] == current_month_symbol]
    
    future_price_df["interest_amount_total"] = (pd.to_numeric(future_price_df["price"], errors="coerce") - int(current_month_df["price"].iloc[0])) * multiplier
    future_price_df["interest_per_day"] = ((future_price_df["interest_amount_total"]) / (pd.to_numeric(future_price_df["days_left"], errors="coerce") - int(current_month_df["days_left"].iloc[0]))).round(1)
    future_price_df["interest_rate"] = (((future_price_df["interest_per_day"] * 365) / (int(current_month_df["price"].iloc[0]) * multiplier)) * 100).round(2)

def get_estimated_todays_price(future_price_df, current_month_symbol):
    avg_per_day_interest = future_price_df.loc[(future_price_df["interest_per_day"] > 100) & (future_price_df["interest_per_day"] < 99999999) & (future_price_df["interest_per_day"].notna()), "interest_per_day"].mean()
    current_month_df = future_price_df[future_price_df["symbol"] == current_month_symbol]
    return current_month_df["price"].iloc[0] - (current_month_df["days_left"].iloc[0] * avg_per_day_interest)

# 3. Gold Mini Futures Analysis

In [7]:
# Gold Mini Futures DF
goldMini_future_symbol_df = get_active_future_symbol_df(GOLD_MINI)
goldMini_future_symbol_df.head()[["exchange_symbol", "symbol_details", "last_update_date"]]

,exchange_symbol,symbol_details,last_update_date
4261,MCX:GOLDM26JANFUT,GOLDM 26 Jan 05 FUT,2025-12-13
8410,MCX:GOLDM26FEBFUT,GOLDM 26 Feb 05 FUT,2025-12-13
12436,MCX:GOLDM26MARFUT,GOLDM 26 Mar 05 FUT,2025-12-13
15370,MCX:GOLDM26APRFUT,GOLDM 26 Apr 03 FUT,2025-12-13
18300,MCX:GOLDM26MAYFUT,GOLDM 26 May 05 FUT,2025-12-13


In [8]:
(goldMini_future_price_df, goldMini_current_month_symbol) = get_combined_future_price_df(goldMini_future_symbol_df)
print("Gold mini Current Month Symbol: ", goldMini_current_month_symbol)

add_analysis_per_day_interest_details(goldMini_future_price_df, goldMini_current_month_symbol, GOLD_MINI_MULTIPLIER)

print("Estimated Today's Gold Price:  ", get_estimated_todays_price(goldMini_future_price_df, goldMini_current_month_symbol))
goldMini_future_price_df

List of Future Symbols:  ['MCX:GOLDM26JANFUT', 'MCX:GOLDM26FEBFUT', 'MCX:GOLDM26MARFUT', 'MCX:GOLDM26APRFUT', 'MCX:GOLDM26MAYFUT', 'MCX:GOLDM26JUNFUT']
Gold mini Current Month Symbol:  MCX:GOLDM26JANFUT
Estimated Today's Gold Price:   120939.31999999999


,symbol,expiry,price,days_left,price_yesterday,volume,spread,price_change_percentage,interest_amount_total,interest_per_day,interest_rate
0,MCX:GOLDM26JANFUT,05-01-2026,132911,21,131645,5815,15,0.96,0,NaN,NaN
1,MCX:GOLDM26FEBFUT,05-02-2026,134795,52,133556,2856,4,0.93,18840,607.7,16.69
2,MCX:GOLDM26MARFUT,05-03-2026,136428,80,135145,657,36,0.95,35170,596.1,16.37
3,MCX:GOLDM26APRFUT,03-04-2026,137794,109,136508,195,57,0.94,48830,554.9,15.24
4,MCX:GOLDM26MAYFUT,05-05-2026,139603,141,138397,72,101,0.87,66920,557.7,15.32
5,MCX:GOLDM26JUNFUT,05-06-2026,140974,172,139418,31,191,1.12,80630,534.0,14.66


# 4. Silver Mini Future Analysis

In [9]:
# Gold Mini Futures DF
silverMini_future_symbol_df = get_active_future_symbol_df(SILVER_MINI)
silverMini_future_symbol_df.head()[["exchange_symbol", "symbol_details", "last_update_date"]]

,exchange_symbol,symbol_details,last_update_date
11291,MCX:SILVERM26FEBFUT,SILVERM 26 Feb 27 FUT,2025-12-12
17142,MCX:SILVERM26APRFUT,SILVERM 26 Apr 30 FUT,2025-12-12
22534,MCX:SILVERM26JUNFUT,SILVERM 26 Jun 30 FUT,2025-12-12
27438,MCX:SILVERM26AUGFUT,SILVERM 26 Aug 31 FUT,2025-12-12
28562,MCX:SILVERM26NOVFUT,SILVERM 26 Nov 30 FUT,2025-12-12


In [10]:
(silverMini_future_price_df, silverMini_current_month_symbol) = get_combined_future_price_df(silverMini_future_symbol_df)
print("Silver Mini Current Month Symbol: ", silverMini_current_month_symbol)

add_analysis_per_day_interest_details(silverMini_future_price_df, silverMini_current_month_symbol, SILVER_MINI_MULTIPLIER)

print("Estimated Today's Silver Price:  ", get_estimated_todays_price(silverMini_future_price_df, silverMini_current_month_symbol))
silverMini_future_price_df



List of Future Symbols:  ['MCX:SILVERM26FEBFUT', 'MCX:SILVERM26APRFUT', 'MCX:SILVERM26JUNFUT', 'MCX:SILVERM26AUGFUT', 'MCX:SILVERM26NOVFUT']
Silver Mini Current Month Symbol:  MCX:SILVERM26FEBFUT
Estimated Today's Silver Price:   168850.1


,symbol,expiry,price,days_left,price_yesterday,volume,spread,price_change_percentage,interest_amount_total,interest_per_day,interest_rate
0,MCX:SILVERM26FEBFUT,27-02-2026,193400,76,199210,138126,110,-2.92,0,NaN,NaN
1,MCX:SILVERM26APRFUT,30-04-2026,197500,138,202388,27037,142,-2.42,20500,330.6,12.48
2,MCX:SILVERM26JUNFUT,30-06-2026,201361,199,205418,5634,398,-1.97,39805,323.6,12.21
3,MCX:SILVERM26AUGFUT,31-08-2026,204700,261,208459,944,755,-1.80,56500,305.4,11.53
4,MCX:SILVERM26NOVFUT,30-11-2026,211753,352,213640,448,407,-0.88,91765,332.5,12.55


In [12]:
!python --version

Python 3.14.2


In [21]:
!/Applications/Python\ 3.14/Install\ Certificates.command

 -- pip install --upgrade certifi
 -- removing any existing file or link
 -- creating symlink to certifi certificate bundle
 -- setting permissions
 -- update complete
